In [ ]:
import os
import shutil

# Define the paths
training_path = 'training'
validation_path = 'validation'
final_path = 'final'

# Create the final directories
# os.makedirs(os.path.join(final_path, 'food'), exist_ok=True)
# os.makedirs(os.path.join(final_path, 'non_food'), exist_ok=True)

# Function to copy and rename files from source to destination
def copy_files(src, dest, prefix):
    for file_name in os.listdir(src):
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            new_file_name = f"{prefix}_{file_name}"
            shutil.copy2(full_file_name, os.path.join(dest, new_file_name))

# Copy and rename food images from training and validation
copy_files(os.path.join(training_path, 'food'), os.path.join(final_path, 'food'), 'training_food')
copy_files(os.path.join(validation_path, 'food'), os.path.join(final_path, 'food'), 'validation_food')

# Copy and rename non_food images from training and validation
copy_files(os.path.join(training_path, 'non_food'), os.path.join(final_path, 'non_food'), 'training_non_food')
copy_files(os.path.join(validation_path, 'non_food'), os.path.join(final_path, 'non_food'), 'validation_non_food')

print("Files have been successfully copied and renamed to the 'final' directory.")


In [ ]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# Define the base path
base_path = "/Users/vrajshah1510/Downloads/archive"
directories = ["evaluation"]
subdirs = ["food","non_food"]

# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.Blur(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5)
])

# Function to augment and save images
def augment_and_save_images(directory):
    for subdir in subdirs:
        subdir_path = os.path.join(directory, subdir)
        if not os.path.exists(subdir_path):
            continue
        output_subdir = os.path.join(subdir_path, "augmented")
        os.makedirs(output_subdir, exist_ok=True)
        
        for image_name in tqdm(os.listdir(subdir_path)):
            image_path = os.path.join(subdir_path, image_name)
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            for i in range(10):
                augmented = augmentation_pipeline(image=image)
                augmented_image = augmented["image"]
                output_image_path = os.path.join(output_subdir, f"{os.path.splitext(image_name)[0]}_aug_{i}.jpg")
                cv2.imwrite(output_image_path, augmented_image)

# Iterate through each main directory and perform augmentation
for directory in directories:
    dir_path = os.path.join(base_path, directory)
    augment_and_save_images(dir_path)

print("Augmentation completed.")
